# Dataset Preprocessing for file size reduction and inferring schema

In [30]:
import polars as pl
import os

## Create datasets dict

In [31]:
datasets_dict = {
    'name' : ['iris', 'mpg', 'titanic', 'starbucks', 'netflix'],
    'source' : [
            'https://www.kaggle.com/datasets/himanshunakrani/iris-dataset',
            'https://www.kaggle.com/datasets/uciml/autompg-dataset',
            'https://www.kaggle.com/competitions/titanic',
            'https://www.kaggle.com/datasets/harshalhonde/starbucks-reviews-dataset',
            'https://www.kaggle.com/datasets/shivamb/netflix-shows'
            ],
    'license' : ['CC0: Public Domain', 'CC0: Public Domain', None, 'CC0: Public Domain', 'CC0: Public Domain'],
    'origin' : ['Kaggle', 'Kaggle', 'Kaggle', 'Kaggle', 'Kaggle']

}

In [32]:

datasets_df = pl.DataFrame(datasets_dict)
datasets_df

name,source,license,origin
str,str,str,str
"""iris""","""https://www.kaggle.com/dataset…","""CC0: Public Domain""","""Kaggle"""
"""mpg""","""https://www.kaggle.com/dataset…","""CC0: Public Domain""","""Kaggle"""
"""titanic""","""https://www.kaggle.com/competi…",null,"""Kaggle"""
"""starbucks""","""https://www.kaggle.com/dataset…","""CC0: Public Domain""","""Kaggle"""
"""netflix""","""https://www.kaggle.com/dataset…","""CC0: Public Domain""","""Kaggle"""


## Load and process datasets

In [33]:
def load_dataset(name):
    return pl.read_csv(f'csv/{name}.csv',infer_schema_length=10000)


In [34]:
iris = load_dataset('iris')
mpg = load_dataset('mpg')
titanic = load_dataset('titanic')
starbucks = load_dataset('starbucks')
netflix = load_dataset('netflix')

In [35]:
size_iris_csv = os.path.getsize('csv/iris.csv')
size_mpg_csv = os.path.getsize('csv/mpg.csv')
size_titanic_csv = os.path.getsize('csv/titanic.csv')
size_starbucks_csv = os.path.getsize('csv/starbucks.csv')
size_netflix_csv = os.path.getsize('csv/netflix.csv')

In [36]:
iris_initial_size = iris.estimated_size()
mpg_initial_size = mpg.estimated_size()
titanic_initial_size = titanic.estimated_size()
starbucks_initial_size = starbucks.estimated_size()
netflix_initial_size = netflix.estimated_size()

In [37]:
size_dict = {
    'name' : ['iris', 'mpg', 'titanic', 'starbucks', 'netflix'],
    'size_csv' : [size_iris_csv, size_mpg_csv, size_titanic_csv, size_starbucks_csv, size_netflix_csv],
    'size_initial' : [iris_initial_size, mpg_initial_size, titanic_initial_size, starbucks_initial_size, netflix_initial_size],
    'size_after_processing' : ['iris', 'mpg', 'titanic', 'starbucks', 'netflix'], # to be overwritten
    'size_parquet_zstd' : ['iris', 'mpg', 'titanic', 'starbucks', 'netflix'], # to be overwritten
    'size_feather_zstd' : ['iris', 'mpg', 'titanic', 'starbucks', 'netflix'], # to be overwritten
}
size_dict

{'name': ['iris', 'mpg', 'titanic', 'starbucks', 'netflix'],
 'size_csv': [3858, 18131, 61194, 26898, 3399671],
 'size_initial': [6050, 29651, 85862, 35697, 3265729],
 'size_after_processing': ['iris', 'mpg', 'titanic', 'starbucks', 'netflix'],
 'size_parquet_zstd': ['iris', 'mpg', 'titanic', 'starbucks', 'netflix'],
 'size_feather_zstd': ['iris', 'mpg', 'titanic', 'starbucks', 'netflix']}

In [38]:
def save_dataframe(df:pl.DataFrame, name:str)->None:
    # get index of datasets
    index = size_dict['name'].index(name)
    
    #write dataframes
    df.write_parquet(f'parquet/{name}.parquet', compression='zstd')
    df.write_ipc(f'feather/{name}.feather', compression='zstd')
    #get sizes for dataframe and binary files
    size_after_processing = df.estimated_size()
    size_parquet_zstd = os.path.getsize(f'parquet/{name}.parquet')
    size_feather_zstd = os.path.getsize(f'feather/{name}.feather')
    # get size of csv
    size_csv = os.path.getsize(f'csv/{name}.csv')
    # update size_dict    
    size_dict['size_after_processing'][index] = size_after_processing
    size_dict['size_parquet_zstd'][index] = size_parquet_zstd
    size_dict['size_feather_zstd'][index] = size_feather_zstd
    #compare compression percentages
    print(f'For {name} dataset maximum compression is achieved by:')
    if size_parquet_zstd > size_feather_zstd:
        print(f'\tFeather')
    else:
        print(f'\tParquet')
    
    #print percentage to inital size
    print(f'Processed:{size_after_processing/size_csv*100:.2f}%\n',
          f'Parquet:{size_parquet_zstd/size_csv*100:.2f}%\n',
          f'Feather:{size_feather_zstd/size_csv*100:.2f}%\n',
          sep='')
    
    return None



### iris

In [39]:
iris.head()

sepal_length,sepal_width,petal_length,petal_width,species
f64,f64,f64,f64,str
5.1,3.5,1.4,0.2,"""setosa"""
4.9,3.0,1.4,0.2,"""setosa"""
4.7,3.2,1.3,0.2,"""setosa"""
4.6,3.1,1.5,0.2,"""setosa"""
5.0,3.6,1.4,0.2,"""setosa"""


In [40]:
iris = iris.with_columns(pl.col(['sepal_length','sepal_width','petal_length','petal_width']).cast(pl.Float32))

In [41]:
save_dataframe(iris, 'iris')

For iris dataset maximum compression is achieved by:
	Feather
Processed:94.61%
Parquet:73.20%
Feather:65.91%



### starbucks

In [42]:
starbucks.head()

Beverage_category,Beverage,Beverage_prep,Calories,Total Fat (g),Trans Fat (g),Saturated Fat (g),Sodium (mg),Total Carbohydrates (g),Cholesterol (mg),Dietary Fibre (g),Sugars (g),Protein (g),Vitamin A (% DV),Vitamin C (% DV),Calcium (% DV),Iron (% DV),Caffeine (mg)
str,str,str,i64,str,f64,f64,i64,i64,i64,i64,i64,f64,str,str,str,str,str
"""Coffee""","""Brewed Coffee""","""Short""",3,"""0.1""",0.0,0.0,0,5,0,0,0,0.3,"""0%""","""0%""","""0%""","""0%""","""175"""
"""Coffee""","""Brewed Coffee""","""Tall""",4,"""0.1""",0.0,0.0,0,10,0,0,0,0.5,"""0%""","""0%""","""0%""","""0%""","""260"""
"""Coffee""","""Brewed Coffee""","""Grande""",5,"""0.1""",0.0,0.0,0,10,0,0,0,1.0,"""0%""","""0%""","""0%""","""0%""","""330"""
"""Coffee""","""Brewed Coffee""","""Venti""",5,"""0.1""",0.0,0.0,0,10,0,0,0,1.0,"""0%""","""0%""","""2%""","""0%""","""410"""
"""Classic Espresso Drinks""","""Caffè Latte""","""Short Nonfat Milk""",70,"""0.1""",0.1,0.0,5,75,10,0,9,6.0,"""10%""","""0%""","""20%""","""0%""","""75"""


TODO with starbukcs dataframe

total fat to f32
trans fat to f32
saturated fat to f32
protein to f32

sugar to u16
calories to u16
dietary fibre to u16
sodium to u16
total carbohydrates to u16

In [43]:
starbucks.columns

['Beverage_category',
 'Beverage',
 'Beverage_prep',
 'Calories',
 ' Total Fat (g)',
 'Trans Fat (g) ',
 'Saturated Fat (g)',
 ' Sodium (mg)',
 ' Total Carbohydrates (g) ',
 'Cholesterol (mg)',
 ' Dietary Fibre (g)',
 ' Sugars (g)',
 ' Protein (g) ',
 'Vitamin A (% DV) ',
 'Vitamin C (% DV)',
 ' Calcium (% DV) ',
 'Iron (% DV) ',
 'Caffeine (mg)']

In [44]:
starbucks = starbucks.with_columns(
    pl.col(['Trans Fat (g) ','Saturated Fat (g)',' Protein (g) ']).cast(pl.Float32),
    pl.col([' Sugars (g)','Calories',' Dietary Fibre (g)',' Sodium (mg)',' Total Carbohydrates (g) ']).cast(pl.UInt16)
)

In [45]:
save_dataframe(starbucks, 'starbucks')

For starbucks dataset maximum compression is achieved by:
	Feather
Processed:94.93%
Parquet:42.17%
Feather:41.27%



### mpg

In [46]:
mpg.head()

mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
f64,i64,f64,str,i64,f64,i64,i64,str
18.0,8,307.0,"""130""",3504,12.0,70,1,"""chevrolet chevelle malibu"""
15.0,8,350.0,"""165""",3693,11.5,70,1,"""buick skylark 320"""
18.0,8,318.0,"""150""",3436,11.0,70,1,"""plymouth satellite"""
16.0,8,304.0,"""150""",3433,12.0,70,1,"""amc rebel sst"""
17.0,8,302.0,"""140""",3449,10.5,70,1,"""ford torino"""


In [47]:
mpg = mpg.with_columns(
    pl.col("horsepower").cast(pl.UInt16,strict=False),
    pl.col(pl.Float64).cast(pl.Float32),
    pl.col(pl.Int64).cast(pl.UInt16)
    )
mpg.head()

mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
f32,u16,f32,u16,u16,f32,u16,u16,str
18.0,8,307.0,130,3504,12.0,70,1,"""chevrolet chevelle malibu"""
15.0,8,350.0,165,3693,11.5,70,1,"""buick skylark 320"""
18.0,8,318.0,150,3436,11.0,70,1,"""plymouth satellite"""
16.0,8,304.0,150,3433,12.0,70,1,"""amc rebel sst"""
17.0,8,302.0,140,3449,10.5,70,1,"""ford torino"""


In [48]:
save_dataframe(mpg, 'mpg')

For mpg dataset maximum compression is achieved by:
	Parquet
Processed:83.90%
Parquet:54.10%
Feather:56.93%



### titanic

In [49]:
titanic = titanic.with_columns(pl.col(pl.Int64).cast(pl.UInt16),
                               pl.col(pl.Float64).cast(pl.Float32)
                               )
titanic.head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
u16,u16,u16,str,str,f32,u16,u16,str,f32,str,str
1,0,3,"""Braund, Mr. Owen Harris""","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. John Bradley (Fl…","""female""",38.0,1,0,"""PC 17599""",71.283302,"""C85""","""C"""
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S"""
4,1,1,"""Futrelle, Mrs. Jacques Heath (…","""female""",35.0,1,0,"""113803""",53.099998,"""C123""","""S"""
5,0,3,"""Allen, Mr. William Henry""","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""


In [50]:
save_dataframe(titanic, 'titanic')

For titanic dataset maximum compression is achieved by:
	Parquet
Processed:84.98%
Parquet:42.81%
Feather:53.75%



### netflix

In [51]:
netflix.head()

show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
str,str,str,str,str,str,str,i64,str,str,str,str
"""s1""","""Movie""","""Dick Johnson Is Dead""","""Kirsten Johnson""",null,"""United States""","""September 25, 2021""",2020,"""PG-13""","""90 min""","""Documentaries""","""As her father nears the end of…"
"""s2""","""TV Show""","""Blood & Water""",null,"""Ama Qamata, Khosi Ngema, Gail …","""South Africa""","""September 24, 2021""",2021,"""TV-MA""","""2 Seasons""","""International TV Shows, TV Dra…","""After crossing paths at a part…"
"""s3""","""TV Show""","""Ganglands""","""Julien Leclercq""","""Sami Bouajila, Tracy Gotoas, S…",null,"""September 24, 2021""",2021,"""TV-MA""","""1 Season""","""Crime TV Shows, International …","""To protect his family from a p…"
"""s4""","""TV Show""","""Jailbirds New Orleans""",null,null,null,"""September 24, 2021""",2021,"""TV-MA""","""1 Season""","""Docuseries, Reality TV""","""Feuds, flirtations and toilet …"
"""s5""","""TV Show""","""Kota Factory""",null,"""Mayur More, Jitendra Kumar, Ra…","""India""","""September 24, 2021""",2021,"""TV-MA""","""2 Seasons""","""International TV Shows, Romant…","""In a city of coaching centers …"


In [52]:
netflix = netflix.with_columns(pl.col(pl.Int64).cast(pl.UInt16))

In [53]:
save_dataframe(netflix, 'netflix')

For netflix dataset maximum compression is achieved by:
	Parquet
Processed:94.51%
Parquet:32.96%
Feather:44.42%



### Finalizing

In [54]:
sizes_df = pl.DataFrame(size_dict)

In [55]:
df = datasets_df.join(sizes_df, on='name')

In [56]:
df.head()

name,source,license,origin,size_csv,size_initial,size_after_processing,size_parquet_zstd,size_feather_zstd
str,str,str,str,i64,i64,i64,i64,i64
"""iris""","""https://www.kaggle.com/dataset…","""CC0: Public Domain""","""Kaggle""",3858,6050,3650,2824,2543
"""mpg""","""https://www.kaggle.com/dataset…","""CC0: Public Domain""","""Kaggle""",18131,29651,15212,9809,10322
"""titanic""","""https://www.kaggle.com/competi…",null,"""Kaggle""",61194,85862,52004,26200,32894
"""starbucks""","""https://www.kaggle.com/dataset…","""CC0: Public Domain""","""Kaggle""",26898,35697,25533,11343,11100
"""netflix""","""https://www.kaggle.com/dataset…","""CC0: Public Domain""","""Kaggle""",3399671,3265729,3212887,1120411,1510030


In [57]:
df = df.with_columns((pl.col('size_parquet_zstd')/pl.col('size_csv')).alias('parquet_size_rate'),
                     (pl.col('size_feather_zstd')/pl.col('size_csv')).alias('feather_size_rate'))

In [58]:
df.write_csv('datasets_info.csv')